In [ ]:
# Code is mostly adapted from: http://thepythoncorner.com/dev/how-to-create-a-watchdog-in-python-to-look-for-filesystem-changes/
# But this link is pretty informational too: https://www.geeksforgeeks.org/create-a-watchdog-in-python-to-look-for-filesystem-changes/
# This link shows you how to combine both Handler() and Observer() classes into one class if you wanted to: https://blog.magrathealabs.com/filesystem-events-monitoring-with-python-9f5329b651c3
# Alternatives to the watchdog module: http://timgolden.me.uk/python/win32_how_do_i/watch_directory_for_changes.html

In [ ]:
import time
from watchdog.observers import Observer
from watchdog.events import PatternMatchingEventHandler
from pathlib import Path
import tensorflow as tf

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime

In [ ]:
# Function to pre-process the image from image_path 
def preprocess_image(image_path):
    IMG_WIDTH = IMG_HEIGHT = 200
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])
    img = img/255.0
    single_preprocessed_img = (np.expand_dims(img,0))
    return single_preprocessed_img

# Do I want to create a Handler() and an Observer() class?

In [ ]:
# configure event handler variables
patterns = "*" # the file patterns we want to handle (in my scenario, I will handle all the files)
ignore_patterns = "" # the patterns that we don’t want to handle
ignore_directories = True # if we want to be notified just for regular files (not for directories) 
case_sensitive = True

# Create the event handler
my_event_handler = PatternMatchingEventHandler(patterns, ignore_patterns, ignore_directories, case_sensitive)

In [ ]:
mask_filepaths_queue = []

# a function that will be used when a file is created in the directory.
def on_created(event):
    p = event.src_path # absolute filepath of the file created
    print(str(p) + ' added to folder')
    mask_filepaths_queue.append(p)
    print(str(p) + ' added to queue')
    if len(mask_filepaths_queue) >=2 :
        print("queue full")
        preprocessed_masks = [preprocess_image(x) for x in mask_filepaths_queue]
        mask_preds = batch_prediction(preprocessed_masks, model)
        print(mask_preds)
        write_classification(mask_filepaths_queue, mask_preds)
        mask_filepaths_queue.clear()

In [ ]:
# specify to the handler that we want the on_created function to be called when the on_created event is raised
my_event_handler.on_created = on_created

In [ ]:
# Create an observer object that will monitor a directory, looking for changes that will be handled by the event handler.
path = r"C:\Users\80203607\git\kenai2\Python\src\Multiclassification\Watchdog_testing_folder" # directory to watch; acquire_mask_folder_filepath()
go_recursively = True # a boolean that allow me to catch all the event that occurs even in the sub directories of my current directory. Not applicable in our case
my_observer = Observer()
my_observer.schedule(my_event_handler, path, recursive=go_recursively)

In [ ]:
# Use This cell only to test importing handler.py, else you can just use all the cells above in the jupyter notebook
import handler
# Create an observer object that will monitor a directory, looking for changes that will be handled by the event handler.
path = r"C:\Users\80203607\git\kenai2\Python\src\Multiclassification\Watchdog_testing_folder" # directory to watch; acquire_mask_folder_filepath()
go_recursively = True # a boolean that allow me to catch all the event that occurs even in the sub directories of my current directory. Not applicable in our case
my_observer = Observer()
my_observer.schedule(handler.Handler(), path, recursive=go_recursively)

In [ ]:
# start the observer thread
my_observer.start()
try:
    while True:
        time.sleep(1)    #What does this sleep do?? 
except KeyboardInterrupt:
    my_observer.stop()
my_observer.join()